## Using Spark SQL to Process Streaming Data

Let us understand how we can use Spark SQL to process streaming data. Here are the typical steps that are involved to process data leveraging Spark SQL to process Streaming Data.
* Read data from the source as a Data Frame.
* Create Temporary View using the Streaming Data Frame.
* Use `spark.sql` to process the data using supported SQL queries. This will result in a Data Frame.
* Finally write to target using `writeStream` on top of Data Frame which contain processed data.

```python
spark.conf.set('spark.sql.shuffle.partitions', '2')

import socket
hostname = socket.gethostname()

log_messages = spark. \
    readStream. \
    format("socket"). \
    option("host", hostname). \
    option("port", 9000). \
    load()

log_messages.createOrReplaceTempView('log_messages')

department_count = spark.sql("""
    SELECT split(split(value, ' ')[6], '/')[2] AS department_name, 
        count(1) AS count
    FROM log_messages
    WHERE split(split(value, ' ')[6], '/')[1] = 'department'
    GROUP BY split(split(value, ' ')[6], '/')[2]
""")

department_count. \
    writeStream. \
    outputMode("complete"). \
    format("console"). \
    option('truncate', 'false'). \
    trigger(processingTime='5 seconds'). \
    start()
```